In [7]:
# Her bruger vi picke til at få den trænede data, og åbner det. Jeg har uploadet det i dokumentet

import pickle
import pandas as pd

with open('kmeans_model.pkl', 'rb') as file:
    kmeans_model = pickle.load(file)

with open('logistic_regression_model.pkl', 'rb') as file:
    logistic_regression_model = pickle.load(file)

af_usa_loaded = pd.read_csv('af_usa_processed.csv')

In [14]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import re
import string

In [15]:
# Load dataset
af_usa_loaded = pd.read_csv('/content/af_usa_processed.csv')

# Definer funktionen til at forbehandle tekst
def preprocess_text(text):
    if isinstance(text, str):
        # Converter til små bogstaver
        text = text.lower()
        # Fjern tegnsætning
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Fjern tal
        text = re.sub(r'\d+', '', text)
        # Fjern ekstra mellemrum
        text = ' '.join(text.split())
    else:
        # Hvis der ikke er tekst i stringen
        text = ''
    return text

In [13]:
# Sikr at kolonnen 'use_cleaned' eksisterer
if 'use_cleaned' not in af_usa_loaded.columns:
    # Antager at kolonnen 'use' eksisterer og skal rengøre
    af_usa_loaded['use_cleaned'] = af_usa_loaded['use'].apply(preprocess_text)

# Brug TfidfVectorizer til at konvertere tekstdata til funktione
vectorizer = TfidfVectorizer(max_features=50)

# Remove or replace np.nan values in 'use_cleaned' column
af_usa_loaded['use_cleaned'] = af_usa_loaded['use_cleaned'].fillna('')

tfidf_matrix = vectorizer.fit_transform(af_usa_loaded['use_cleaned'])

# Opret en ny kolonne der angiver, om lån har høj sandsynlighed (1) eller lav sandsynlighed (0)
af_usa_loaded['high_chance'] = np.where(af_usa_loaded['lender_count'] >= af_usa_loaded['lender_count'].median(), 1, 0)

In [16]:
# Definer funktionerne og målene til den superviserede model
X_supervised = pd.DataFrame(tfidf_matrix.toarray())
y_supervised = af_usa_loaded['high_chance']

# Split data i trænings- og testdata
X_train, X_test, y_train, y_test = train_test_split(X_supervised, y_supervised, test_size=0.2, random_state=42)

# Vi bruger logistisk regression model
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train, y_train)

# Funktion til at anbefale baseret på nøgleord
def recommend_chance_supervised(use_description):
    use_description_cleaned = preprocess_text(use_description)
    input_tfidf = vectorizer.transform([use_description_cleaned]).toarray()
    predicted_chance = classifier.predict(input_tfidf)[0]
    return "High Chance" if predicted_chance == 1 else "Low Chance"

In [18]:
# Recommender based on K-Means clustering
def find_similar_loans(loan_input):
    # Definer og tilpas scaler
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(af_usa_loaded[['loan_amount', 'term_in_months', 'lender_count']])

    # Definer og tilpas PCA
    pca = PCA(n_components=2)
    pca_features = pca.fit_transform(scaled_features)

    # Find nærmeste klynge baseret på inputdata
    input_data = [[loan_input, af_usa_loaded['term_in_months'].mean(), af_usa_loaded['lender_count'].mean()]]
    input_scaled = scaler.transform(input_data)
    input_pca = pca.transform(input_scaled)
    input_cluster = kmeans_model.predict(input_pca)[0]

    # Anbefalinger fra samme klynge
    recommended_loans = af_usa_loaded[af_usa_loaded['cluster_kmeans'] == input_cluster].head(5)
    return recommended_loans[['loan_amount', 'term_in_months', 'lender_count', 'sector', 'activity']]

In [19]:
# Eksempel på brug af funktionerne
loan_description = "purchase of equipment for business"
predicted_chance = recommend_chance_supervised(loan_description)
similar_loans = find_similar_loans(5000)

(predicted_chance, similar_loans)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


('High Chance',
    loan_amount  term_in_months  lender_count       sector         activity
 1       4000.0            26.0            93       Retail  Cosmetics Sales
 3       5000.0            32.0           158  Agriculture          Farming
 4       5450.0            32.0           164     Services         Services
 6       5450.0            32.0           166       Retail           Retail
 8       5000.0            24.0           150         Food       Restaurant)